### **載入資料**
資料來源:[TIMIT](https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3)

從google drive下載資料並解壓縮，應該要得到timit_11資料夾並包括以下檔案:
*  train_11.npy     (訓練資料)
*  train_label_11.npy  (訓練標籤)
*  test_11.npy     (測試資料)



In [3]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
# !unzip data.zip
# !ls

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
100% 372M/372M [00:04<00:00, 78.9MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  drive  sample_data  timit_11


### **準備資料**
從.npy檔載入訓練跟測試資料

In [55]:
import numpy as np

print('Loading data ...')

# data_root = './timit_11/'
train = np.load('train_11.npy')
train_label = np.load('train_label_11.npy')
test = np.load('test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


### **創建資料集**

In [56]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()      # 將資料轉成浮點數並創建張量tensor
        if y is not None:
            y = y.astype(int)            # 將label轉成int
            self.label = torch.LongTensor(y)      # 將label轉成tensor儲存
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]   # 一次取一筆訓練資料
        else:
            return self.data[idx]            # 一次取一筆測試資料

    def __len__(self):
        return len(self.data)

### **訓練資料分割**
將訓練資料分成訓練資料集跟驗證資料集，並用VAL_RATIO控制兩個資料集的比例

In [57]:
VAL_RATIO = 0.1     # 控制兩個資料集的比例

percent = int(train.shape[0] * (1 - VAL_RATIO))     # 計算訓練資料集的長度
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]    # 根據長度對資料集進行分割
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


### **創建DataLoader**
用Dataset創建DataLoader，並在這邊調整batch size


In [58]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128    # 調整batch size大小

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
# 只在training開啟洗牌功能(shuffle)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

### **清理不需要的變數**
這次的資料集大小相當巨大，因此要要注意Colab記憶體的狀況

透過清理不需要的變數來節省記憶體的空間，如果還需要在這個block被清理掉的變數，則將這個block刪掉或是晚點再清理變數

參考資料:https://docs.python.org/zh-tw/3/library/gc.html

In [59]:
import gc
# 用 del 清除以下變數(資料已存進dataset)
del train, train_label, train_x, train_y, val_x, val_y
gc.collect()     # 清除記憶體中不可訪問(未引用)的資料

2506

### **創建 Model**
定義model的結構，並嘗試拿到好的分數

In [61]:
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # self.layer1 = nn.Linear(429, 1024)
        # self.layer2 = nn.Linear(1024, 512)
        # self.layer3 = nn.Linear(512, 128)
        # self.out = nn.Linear(128, 39)

        # self.act_fn = nn.Sigmoid()
        self.net = nn.Sequential(
            nn.Linear(429, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(p=0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 39),
        )

    def forward(self, x):
        # x = self.layer1(x)
        # x = self.act_fn(x)

        # x = self.layer2(x)
        # x = self.act_fn(x)

        # x = self.layer3(x)
        # x = self.act_fn(x)

        # x = self.out(x)

        # return x
        return self.net(x).squeeze(1)

### **Training**

In [62]:
# 確認設備
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

固定random seeds以確保在不更改參數的情況下，model的輸出不會改變

參考資料:https://zhuanlan.zhihu.com/p/161575780

In [64]:
def same_seeds(seed):
    torch.manual_seed(seed)     # 為CPU設定random seed
    if torch.cuda.is_available():
        # torch.cuda.manual_seed(seed)    # 為特定GPU設定random seed
        torch.cuda.manual_seed_all(seed)  # 為所有GPU設定random seed
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False    # 最佳化卷積層
    torch.backends.cudnn.deterministic = True   # 讓GPU的輸出一致

調整訓練參數

In [66]:
# 設定random seed
same_seeds(0)

# 抓取設備
device = get_device()
print(f'DEVICE: {device}')

# 訓練參數
num_epoch = 60
learning_rate = 0.0001

# 檢查點的儲存路徑
model_path = './model.ckpt'

# 創建模型，定義loss function與最佳化方法
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0005)

DEVICE: cuda


開始訓練

In [67]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train()   # 將model設成training model
    for i, data in enumerate(train_loader):
        inputs, labels = data                    # 從train_loader載入訓練input跟label
        inputs, labels = inputs.to(device), labels.to(device)   # 將input跟label複製到device中進行運算
        optimizer.zero_grad()                     # 將gradient設為0(避免梯度累積)
        outputs = model(inputs)                   # 計算輸出(forward pass)
        batch_loss = criterion(outputs, labels)           # 計算cross-entropy loss
        _, train_pred = torch.max(outputs, 1)           # 找出outputs中最大值的位置(索引值)(0為列，1為行)
        batch_loss.backward()                     # 計算gradient(backpropagation)
        optimizer.step()                        # 更新參數

        # 比較每筆training data的train_pred跟labels的值，相同回傳1不同回傳0
        # 將train_pred.cpu() == labels.cpu()的值加總(越大越好)，再用item()讀取tensor值並回傳float
        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        # 計算1個batch的loss
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval()                          # 將model設成evaluation model
        with torch.no_grad():                     # 關掉梯度計算
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1)     # 找出機率最高的class的索引值

                val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
                val_loss += batch_loss.item()

            # len(train_set)是指training data的數量，len(train_loader)是指batch的數量(len(train_set)/batch_size)
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # 如果model在驗證時的效果比較好，儲存該model的checkpoint
            # model.state_dict()是用來查看model的參數(通常用於儲存model)
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

[001/060] Train Acc: 0.547156 Loss: 1.552408 | Val Acc: 0.665878 loss: 1.070760
saving model with acc 0.666
[002/060] Train Acc: 0.626883 Loss: 1.204728 | Val Acc: 0.691139 loss: 0.970993
saving model with acc 0.691
[003/060] Train Acc: 0.648854 Loss: 1.121065 | Val Acc: 0.707221 loss: 0.913175
saving model with acc 0.707
[004/060] Train Acc: 0.662427 Loss: 1.069816 | Val Acc: 0.713734 loss: 0.884824
saving model with acc 0.714
[005/060] Train Acc: 0.671913 Loss: 1.034243 | Val Acc: 0.722035 loss: 0.857114
saving model with acc 0.722
[006/060] Train Acc: 0.679157 Loss: 1.006031 | Val Acc: 0.726361 loss: 0.838246
saving model with acc 0.726
[007/060] Train Acc: 0.686898 Loss: 0.980815 | Val Acc: 0.728377 loss: 0.828933
saving model with acc 0.728
[008/060] Train Acc: 0.691589 Loss: 0.962700 | Val Acc: 0.733678 loss: 0.810986
saving model with acc 0.734
[009/060] Train Acc: 0.696736 Loss: 0.944851 | Val Acc: 0.735459 loss: 0.804998
saving model with acc 0.735
[010/060] Train Acc: 0.70003

### **Testing**
創建測試資料集並從checkpoint載入訓練好的model

In [68]:
# 創建test_set與test_loader
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# 創建模型並從checkpoint載入模型參數
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

開始進行testing

In [107]:
test_len = 451552                  # testing data的長度
predict = []                       # 儲存預測結果
ori_output = []                    # 儲存模型預測的機率分布
model.eval()                        # 將model設成evaluation model
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1)  # 找出機率最高的class的索引值

        # 將預測結果儲存起來
        for y in test_pred.cpu().numpy():
            predict.append(y)
        # 將預測的機率分布儲存起來
        for output in outputs.cpu().numpy():
            ori_output.append(output)
        

ori_output = np.array(ori_output)             # 將ori_output從list轉成array

<class 'list'>
<class 'numpy.ndarray'>


將預測結果存成.csv檔，並將prediction.csv下載下來，之後放上kaggle進行評分

In [106]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

### **Post-processing**

載入訓練標籤

In [74]:
print('Loading data...')

train_label = np.load('train_label_11.npy')

print('Size of training label: {}'.format(train_label.shape))

Loading data...
Size of training label: (1229932,)


創建轉換機率與輸出機率  
轉換機率(transition_probability): 由label i轉成label j的機率分佈    
輸出機率(emission_probability): 輸出為label j的機率分佈

In [93]:
# transition_probability
trans_table = np.zeros((39,39))                     # 創建轉換矩陣(由phoneme i轉成j的機率分佈)
train_label = train_label.astype('int')             # 將train_label換成整數型態

# 累計train_label裡phoneme i轉成j的次數並儲存在trans_table
for i in range(len(train_label)-1):
    trans_table[train_label[i], train_label[i+1]] += 1

# np.sum(trans_table,axis=1,keepdims=True) 是將trans_table的每一行相加，並以一列表示
# 對每行做歸一化，代表第i個label轉換成其他label的機率
trans_table_norm = trans_table / np.sum(trans_table,axis=1,keepdims=True)
# 避免trans_table裡有0
trans_table_norm += 1e-17
# 做log運算後，之後進行連乘運算時值才不會變0
trans_table_norm = np.log(trans_table_norm)

# emission_probability
# 對ori_output每行做Softmax後再做log運算
m = nn.Softmax(dim=1)
test_softmax = m(torch.tensor(ori_output))
test_softmax = np.array(test_softmax)
test_softmax  += 1e-17
test_softmax = np.log(test_softmax)

定義初始狀態並計算label的路徑

In [112]:
tracking = np.zeros((test_len,39))          # 創建label路徑
last_state = test_softmax[0]                # 定義初始狀態

# 計算上一個label為i而現在label為j的機率分佈
for i in range(1,len(test_softmax)):
    # prob = 上一個為label i的機率分佈 * label i變label j的機率分佈 * 現在為label j的機率分佈(log所以用相加表示) 
    prob = last_state.reshape(39,1) + trans_table_norm + test_softmax[i]
    cur_state = np.max(prob, axis=0)                    # 找出每行最大的機率(label j的機率分佈)
    tracking[i] = np.argmax(prob, axis=0)               # 找出每行最大機率的位置(上一個label為多少時label j會有最大的發生機率，也就是label的路徑)
    last_state = cur_state                              # 更新狀態


[-462185.0137989  -462179.73494978 -462184.54895879 -462187.16346367
 -462177.25664997 -462184.10637362 -462184.81400522 -462182.29945183
 -462186.3416976  -462185.1853327  -462186.17600132 -462185.58174254
 -462183.36689259 -462180.07085376 -462181.50840468 -462182.922682
 -462182.36498206 -462179.68008677 -462180.30895534 -462180.94951256
 -462185.41301476 -462184.31305956 -462180.43638587 -462179.52898017
 -462178.50471097 -462171.81888086 -462191.48359653 -462173.68642664
 -462176.80908944 -462172.885737   -462170.47952656 -462182.8765192
 -462182.85583908 -462182.14570166 -462181.2969111  -462181.80879128
 -462186.26203145 -462179.20359971 -462171.38417044]
[36. 36. 36. 36. 36. 36. 36. 36. 36. 36. 38. 36. 36. 36. 36. 36. 36. 36.
 36. 36. 36. 36. 38. 36. 38. 38.  1. 38. 36. 36. 38. 36. 36. 36. 36. 36.
 36. 36. 36.]


In [114]:
pred_ls = [np.argmax(ori_output[-1])]                   # 取出最後一個資料最有可能發生的label(class)放進預測路徑

# 由最後一筆資料的label回推所有所有資料的label
for i in range(0,test_len-1):
    # int(pred_ls[-1]) 為現在這筆資料的label
    # back為前一筆資料的label，而選擇back會使現在label的發生機率為最大
    back = tracking[test_len-i-1][int(pred_ls[-1])]     
    pred_ls.append(int(back))       # 將back放進預測路徑

prediction = pred_ls[::-1]          # 因為是從最後一筆資料回推路徑，所以在輸出結果時要倒過來

[-3.741856   -0.049503   -4.79135    -6.6967063   2.9064243  -2.4792275
 -2.8991768  -2.3663008  -5.1771746  -2.835186   -3.7633338  -3.862223
 -2.2344594  -0.3619265  -1.171139   -1.7299886  -1.9388629   1.0851632
  0.8192001   0.84921896 -1.1757983  -2.7342095   0.7684252   1.14719
  2.1181254   7.064598   -1.7585232   5.075466    3.5843914   5.1907206
  5.452552   -0.1408647   1.058822    0.2669655   2.0648365  -2.5915005
 -4.509722    3.2090673   4.915232  ]
[36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37, 37, 37, 37, 37, 37, 37, 37, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 38, 38, 38, 38, 15, 15, 15, 15, 15, 17, 17, 17, 17, 17, 17, 38, 38, 38, 38, 38, 38, 25, 25, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 14, 14, 14, 14, 14, 14, 38, 38, 38, 38, 38, 38, 38, 30, 30, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 38, 38, 38, 1, 1, 1, 1, 1, 1, 1, 1, 19, 19, 19, 19, 19, 19, 19, 19, 19, 38, 38, 38, 38, 38, 27, 27, 27, 1

In [103]:
with open('pred.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(prediction):
        f.write('{},{}\n'.format(i, y))